## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from torchmetrics.functional.classification import *
from skorch import NeuralNetClassifier
import tensorflow as tf
import time
from keras.models import Sequential
from keras.layers import Dense, InputLayer, Dropout, InputSpec
from keras.utils import *
from keras.layers import *
from keras.models import *
from keras.callbacks import *
from skmultilearn.ext import Keras
from scipy.sparse import lil_matrix

## Read Data

In [2]:
data = pd.read_csv('D:\Project\project-ednn\Code\Final_dataset_Diabetes_Complication.csv')
data.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X24,X25,X26,Y1,Y2,Y3,Y4,Y5,Y6,Y7
0,3,1,1,1,1,1,1,1,3,1,...,1,3,1,2,2,1,2,2,1,2
1,5,3,1,1,2,4,2,1,5,1,...,1,2,2,2,2,2,2,2,1,1
2,4,1,1,1,2,1,2,2,2,2,...,2,3,1,2,2,2,2,2,2,1
3,4,1,1,1,2,4,2,1,1,1,...,2,3,2,2,2,2,1,2,1,2
4,2,1,1,3,1,3,1,1,1,1,...,1,3,1,2,2,2,2,2,2,2


In [3]:
data = data.drop_duplicates(ignore_index=True)
data

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X24,X25,X26,Y1,Y2,Y3,Y4,Y5,Y6,Y7
0,3,1,1,1,1,1,1,1,3,1,...,1,3,1,2,2,1,2,2,1,2
1,5,3,1,1,2,4,2,1,5,1,...,1,2,2,2,2,2,2,2,1,1
2,4,1,1,1,2,1,2,2,2,2,...,2,3,1,2,2,2,2,2,2,1
3,4,1,1,1,2,4,2,1,1,1,...,2,3,2,2,2,2,1,2,1,2
4,2,1,1,3,1,3,1,1,1,1,...,1,3,1,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543770,1,1,2,1,1,4,1,1,5,2,...,2,3,1,2,2,2,2,2,2,2
543771,2,1,1,1,1,4,1,1,5,1,...,1,2,2,2,2,1,2,2,2,1
543772,1,2,1,1,1,3,1,1,2,1,...,1,3,1,2,2,2,2,2,2,1
543773,3,1,2,1,1,3,1,1,2,2,...,1,3,2,2,2,2,2,1,2,2


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 543775 entries, 0 to 543774
Data columns (total 33 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   X1      543775 non-null  int64
 1   X2      543775 non-null  int64
 2   X3      543775 non-null  int64
 3   X4      543775 non-null  int64
 4   X5      543775 non-null  int64
 5   X6      543775 non-null  int64
 6   X7      543775 non-null  int64
 7   X8      543775 non-null  int64
 8   X9      543775 non-null  int64
 9   X10     543775 non-null  int64
 10  X11     543775 non-null  int64
 11  X12     543775 non-null  int64
 12  X13     543775 non-null  int64
 13  X14     543775 non-null  int64
 14  X15     543775 non-null  int64
 15  X16     543775 non-null  int64
 16  X17     543775 non-null  int64
 17  X18     543775 non-null  int64
 18  X19     543775 non-null  int64
 19  X20     543775 non-null  int64
 20  X21     543775 non-null  int64
 21  X22     543775 non-null  int64
 22  X23     543775 non-n

## Data Visualization

In [5]:
data_feature = data.iloc[:, :26]
data_label = data.iloc[:, 26:]
data_label

,Y1,Y2,Y3,Y4,Y5,Y6,Y7
0,2,2,1,2,2,1,2
1,2,2,2,2,2,1,1
2,2,2,2,2,2,2,1
3,2,2,2,1,2,1,2
4,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...
543770,2,2,2,2,2,2,2
543771,2,2,1,2,2,2,1
543772,2,2,2,2,2,2,1
543773,2,2,2,2,1,2,2


## Data Preprocessing

In [6]:
scaler = MinMaxScaler()
scaler.fit(data)

# Transform the DataFrame to obtain the normalized data
data_normalized = scaler.transform(data)

In [7]:
X = data_normalized[:, :26]
y = data_normalized[:, 26:]

In [8]:
data_normalize = pd.DataFrame(scaler.transform(data), columns=data.columns)
df_feature = data_normalize.iloc[:, :26]
df_label = data_normalize.iloc[:, 26:]

## Modelling

In [9]:
weight_list = []
for i, label in enumerate(df_label.columns):
    values_1 = df_label[label][df_label[label] == 1]
    values_2 = df_label[label][df_label[label] == 2]
    dist = 1 - (len(values_1) / len(df_label))
    weight_list.append(dist)

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [11]:
weights_ = torch.Tensor(weight_list).to(device)
weights_

tensor([0.0483, 0.0652, 0.0605, 0.0508, 0.1008, 0.3895, 0.4290],
       device='cuda:0')

In [12]:
def reset_weights(m):
    for layer in m.children():
        if hasattr(layer, 'reset_parameters'):
            # print(f'Reset trainable parameters of layer = {layer}')
            layer.reset_parameters()

In [13]:
class DNNModelwithWeightedDropout(tf.keras.Model):
    def __init__(self, num_features=26, num_classes=7, p=[0.05, 0.01, 0.003]):
        super(DNNModelwithWeightedDropout, self).__init__()
        self.fcn1 = tf.keras.Sequential([
            tf.keras.layers.Dense(32, activation='relu', input_shape=(num_features,)),
            tf.keras.layers.Dense(len(p), activation='softmax')
        ])
        self.dropout = [tf.keras.layers.Dropout(rate=rate) for rate in p]
        self.fcn2 = tf.keras.Sequential([
            tf.keras.layers.Dense(num_classes, activation='sigmoid')
        ])

    def call(self, x, training=None, mask=None):
        x = self.fcn1(x)
        weights = x

        # Apply weighted dropout
        x = [dropout(x[:, i], training=training) * weights[:, i] for i, dropout in enumerate(self.dropout)]

        x = tf.stack(x, axis=1)
        x = self.fcn2(x)
        return x

In [14]:
class DNNDropout(tf.keras.Model):
    def __init__(self, num_features=26, num_classes=7, p=0.5):
        super(DNNDropout, self).__init__()

        self.fcn1 = tf.keras.Sequential([
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(8, activation='relu'),
            tf.keras.layers.Dense(3, activation='softmax')
        ])

        self.dropout1 = tf.keras.layers.Dropout(p)

        self.fcn2 = tf.keras.Sequential([
            tf.keras.layers.Dense(num_classes, activation='sigmoid')
        ])

    def call(self, x, training=False):
        x = self.fcn1(x)
        x = self.dropout1(x)
        x = self.fcn2(x)
        return x

In [15]:
def DropoutModel(n_inputs, n_outputs, dropout_rate=0.3):
    model = DNNDropout(num_features=n_inputs, num_classes=n_outputs, p=dropout_rate)
    lossWeights = weight_list
    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.003), metrics=['accuracy'], loss_weights=lossWeights)

    return model


In [16]:
def ProposedModel1(n_inputs, n_outputs, dropout_rate=[0.3]*3):
    model = DNNModelwithWeightedDropout(num_features=n_inputs, num_classes=n_outputs, p=dropout_rate)
    lossWeights = weight_list
    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.003), metrics=['accuracy'], loss_weights=lossWeights)

    return model

## Standard Training

In [17]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [18]:
dropout_list = [0.1, 0.2, 0.3, 0.5, 0.7, 0.9]
channel_list = [3, 6, 8, 16, 32, 64]

In [19]:
for i in dropout_list:
    base = DropoutModel(26, 7, dropout_rate=i)
    prop = ProposedModel1(26, 7, dropout_rate=[i]*3)

    csv_logger = tf.keras.callbacks.CSVLogger(f'./HyperparameterResults/base_dropout{i}.csv', append=True, separator=',')
    csv_logger2 = tf.keras.callbacks.CSVLogger(f'./HyperparameterResults/prop_dropout{i}.csv', append=True, separator=',')

    base_init = time.time()
    base.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=32, verbose=1, callbacks=[csv_logger])
    base_time = time.time() - base_init
    print(f'Base Model: {base_time}')
    
    prop_init = time.time()
    prop.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=32, verbose=1, callbacks=[csv_logger2])
    prop_time = time.time() - prop_init
    print(f'Proposed Model: {prop_time}')


Epoch 1/10
13595/13595 [==============================] - 55s 4ms/step - loss: 0.0156 - accuracy: 0.3380 - val_loss: 0.0149 - val_accuracy: 0.2240
Epoch 2/10
13595/13595 [==============================] - 53s 4ms/step - loss: 0.0153 - accuracy: 0.2994 - val_loss: 0.0148 - val_accuracy: 0.3229
Epoch 3/10
13595/13595 [==============================] - 52s 4ms/step - loss: 0.0153 - accuracy: 0.2970 - val_loss: 0.0149 - val_accuracy: 0.4424
Epoch 4/10
13595/13595 [==============================] - 52s 4ms/step - loss: 0.0153 - accuracy: 0.3032 - val_loss: 0.0149 - val_accuracy: 0.2613
Epoch 5/10
13595/13595 [==============================] - 52s 4ms/step - loss: 0.0153 - accuracy: 0.2992 - val_loss: 0.0149 - val_accuracy: 0.3566
Epoch 6/10
13595/13595 [==============================] - 52s 4ms/step - loss: 0.0153 - accuracy: 0.3044 - val_loss: 0.0149 - val_accuracy: 0.1592
Epoch 7/10
13595/13595 [==============================] - 52s 4ms/step - loss: 0.0153 - accuracy: 0.3099 - val_loss: 0

In [20]:
for i in channel_list:
    prop = ProposedModel1(26, 7, dropout_rate=[0.3]*i)

    csv_logger2 = tf.keras.callbacks.CSVLogger(f'./HyperparameterResults/prop_channel{i}.csv', append=True, separator=',')
    
    prop_init = time.time()
    prop.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=32, verbose=1, callbacks=[csv_logger2])
    prop_time = time.time() - prop_init
    print(f'Proposed Model: {prop_time}')

Epoch 1/10
13595/13595 [==============================] - 55s 4ms/step - loss: 0.0162 - accuracy: 0.4131 - val_loss: 0.0154 - val_accuracy: 0.4010
Epoch 2/10
13595/13595 [==============================] - 55s 4ms/step - loss: 0.0158 - accuracy: 0.3954 - val_loss: 0.0153 - val_accuracy: 0.3836
Epoch 3/10
13595/13595 [==============================] - 55s 4ms/step - loss: 0.0158 - accuracy: 0.3977 - val_loss: 0.0152 - val_accuracy: 0.1828
Epoch 4/10
13595/13595 [==============================] - 55s 4ms/step - loss: 0.0158 - accuracy: 0.3954 - val_loss: 0.0153 - val_accuracy: 0.2931
Epoch 5/10
13595/13595 [==============================] - 55s 4ms/step - loss: 0.0158 - accuracy: 0.3810 - val_loss: 0.0153 - val_accuracy: 0.4091
Epoch 6/10
13595/13595 [==============================] - 56s 4ms/step - loss: 0.0158 - accuracy: 0.3872 - val_loss: 0.0154 - val_accuracy: 0.3351
Epoch 7/10
13595/13595 [==============================] - 56s 4ms/step - loss: 0.0158 - accuracy: 0.3950 - val_loss: 0